# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Scrape events from a website
* Follow links to those events to retrieve further information
* Clean and store scraped data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [1]:
#Load the https://www.residentadvisor.net/events page in your browser.

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [2]:
#Open the inspect element feature in your browser

## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [3]:
import re
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
def scrape_events(events_page_url):
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    event_listings = soup.find('div', id="event-listing")
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text[:-2]
        else:
            continue
    df = pd.DataFrame(rows)
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

## Write a Function to Retrieve the URL for the Next Page

In [5]:
def next_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    isNext = soup.find('a', attrs={'ga-event-action':"Next "})
    if isNext:
        url_ext = isNext.attrs['href']
        next_page_url = "https://www.residentadvisor.net" + url_ext
    else:
        next_page_url = None
    return next_page_url

## Scrape the Next 100 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [6]:
dfs = []
total_rows = 0
cur_url = 'https://www.residentadvisor.net/events/us/sanfrancisco'
while total_rows <= 100 and cur_url:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
    time.sleep(.2)
df = pd.concat(dfs)
df = df.reset_index(drop=True)
print(len(df))
df

102


,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Werd. Residents and Friends,Monarch,"Sun, 07 Jul 2019",4.0
1,"Permission to Land: Dragonfly, Jef Stott Speci...",The Midway,"Sun, 07 Jul 2019",2.0
2,Stamina Sundays:8 Year Anniversary,F8 1192 Folsom,"Sun, 07 Jul 2019",2.0
3,"After Dawn with Papa Lu, Elz and Bryan Boogie",Monroe,"Sun, 07 Jul 2019",2.0
4,Tutu Tuesday feat. Groovewell Djs,Harlot,"Tue, 09 Jul 2019",2.0
5,"Housepitality: Slope 114, Gonzo, Miguel Solari...",F8 1192 Folsom,"Wed, 10 Jul 2019",2.0
6,Supernature Thursdays: Alinka - DJ M3 aka Make...,Monarch,"Thu, 11 Jul 2019",2.0
7,"Supervixen 013: Jessie Calistri, Smokes and Ca...",Underground SF,"Thu, 11 Jul 2019",3.0
8,Substance with Rescue,The EndUp,"Thu, 11 Jul 2019",2.0
9,Kelly Lee Owens (Free with Rsvp),1015 Folsom,"Thu, 11 Jul 2019",2.0


## Summary 

Congratulations! In this lab, you successfully scraped a website for concert event information!